In [1044]:
import pandas as pd

from  colorama  import  Fore
import os

from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb




## Dataset

In [1045]:
# Leer el csv y sacar por pantalla las cinco primeras filas.
df = pd.read_csv('../data/df_model_train.csv')

df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'ShoppingMall', 'FoodCourt', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [1046]:
df.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,ShoppingMall,FoodCourt,Spa,VRDeck,Name,Transported
0,0003_01,Europa,False,A/0/S,TRAPPIST-1e,0.734177,True,0.395423,0.000000,0.794185,0.879710,0.387660,Susent,False
1,0003_02,Europa,False,A/0/S,TRAPPIST-1e,0.417722,False,0.000000,0.588099,0.694741,0.809679,0.522016,Susent,False
2,0056_01,Europa,False,A/1/S,TRAPPIST-1e,0.025316,False,0.000000,0.000000,0.000000,0.000000,0.000000,Tractive,True
3,0056_02,Europa,True,A/1/S,TRAPPIST-1e,0.481013,False,0.000000,0.000000,0.000000,0.000000,0.000000,Tractive,True
4,0056_03,Europa,False,A/1/S,TRAPPIST-1e,0.341772,False,0.588799,0.807212,0.621862,0.564975,0.137374,Tractive,True


In [1047]:
# Asignar X eliminando la columna objetivo 'Transported'
X = df.drop(columns=['Transported'])

# Asegurándonos de que las variables categóricas estén codificadas como 'category'
categorical_columns = X.select_dtypes(include=['object']).columns  # Selecciona columnas categóricas

for col in categorical_columns:
    X[col] = X[col].astype('category')

# Variable objetivo (y)
y = df['Transported'].astype(int)

# Reemplazar espacios en los nombres de las características (por si hay alguno)
X.columns = X.columns.str.replace(' ', '_')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Modelos buenos

In [1048]:
# Inicializar el modelo LGBMClassifier
lgb_model = lgb.LGBMClassifier(
    n_estimators=1000,  # Un número alto de árboles para dar espacio al early stopping
    learning_rate=0.01,
    max_depth=5,
    subsample=1.0,
    num_leaves=37,
    colsample_bytree=1.0,
    random_state=42,
     max_bin=2550
)

# Entrenar el modelo con early stopping usando callbacks
callbacks = [
    lgb.early_stopping(stopping_rounds=50),  # Early stopping después de 50 iteraciones sin mejora
]

lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],  # Conjunto de validación
    eval_metric='logloss',  # Métrica a optimizar en el conjunto de validación
    callbacks=callbacks,  # Pasar el callback para early stopping
)

# Predecir en el conjunto de validación
y_pred = lgb_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy en conjunto de validación: {accuracy}')


[LightGBM] [Info] Number of positive: 3498, number of negative: 3456
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000483 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4234
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503020 -> initscore=0.012080
[LightGBM] [Info] Start training from score 0.012080
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [1049]:
# Crear el modelo XGBClassifier con los parámetros dados
xgb_model = xgb.XGBClassifier(
    eval_metric='logloss',      # Métrica para clasificación
    n_estimators=100,           # Número de árboles (iteraciones)
    learning_rate=0.05,         # Tasa de aprendizaje
    subsample=0.8,              # Subconjunto de muestras utilizadas para cada árbol
    colsample_bytree=0.6,       # Subconjunto de características utilizadas para construir cada árbol
    min_child_weight=7,         # Tamaño mínimo del nodo hijo
    random_state=42,            # Semilla para reproducibilidad
    reg_lambda=2,               # Regularización L2
    reg_alpha=1,                # Regularización L1
    max_depth=5,                # Profundidad máxima del árbol
    gamma=0.1,                   # Reducción mínima en la métrica de pérdida para hacer un split
    enable_categorical=True     # Habilitar soporte para variables categóricas
)

# Entrenar el modelo
xgb_model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test)

# Evaluar el rendimiento con una métrica como la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.7930


# Usando tecnicas de ensamble

In [1050]:
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score


# Inicializar el VotingClassifier con LightGBM y XGBoost
voting_clf = VotingClassifier(
    estimators=[('lightgbm', lgb_model), ('xgboost', xgb_model)],
    voting='soft'  # Usamos votación blanda para promediar probabilidades
)

# Entrenar el VotingClassifier
voting_clf.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred = voting_clf.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy del Voting Classifier en conjunto de validación: {accuracy}')


[LightGBM] [Info] Number of positive: 3498, number of negative: 3456
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000546 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4234
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503020 -> initscore=0.012080
[LightGBM] [Info] Start training from score 0.012080
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

# Otros Modelos

## Random Forest

## Ajuste de hiperparámetros con XGBoost usando RandomizedSearchCV

In [1051]:
# from sklearn.model_selection import RandomizedSearchCV
# from xgboost import XGBClassifier

# # Define the hyperparameter grid for RandomizedSearchCV
# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
#     'max_depth': [3, 5, 7, 10],
#     'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0],
#     'gamma': [0, 1, 5],
#     'min_child_weight': [1, 3, 5, 7]
# }

# # Initialize the XGBoost model
# xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# # Set up RandomizedSearchCV
# random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid,
#                                        n_iter=10, cv=3, verbose=1, random_state=42, n_jobs=-1)

# # Fit the RandomizedSearchCV to the data
# random_search_xgb.fit(X_train, y_train)

# # Retrieve the best parameters and the best model from RandomizedSearchCV
# best_params_xgb = random_search_xgb.best_params_
# best_model_xgb = random_search_xgb.best_estimator_

# # Evaluate the best XGBoost model on the test set
# y_pred_xgb = best_model_xgb.predict(X_test)
# best_xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
# best_xgb_report = classification_report(y_test, y_pred_xgb)

# print(best_params_xgb)
# print(best_xgb_accuracy)
# print(best_xgb_report)


# Datos de prueba

In [1052]:
# Leer el csv y sacar por pantalla las cinco primeras filas.
df_test = pd.read_csv('../data/df_model_test.csv')

df_test.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'ShoppingMall', 'FoodCourt', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [1053]:
df_test.head()

df_test.shape

(4277, 14)

In [1054]:
# Asignar X eliminando la columna objetivo 'Transported'
X_to_test = df_test.drop(columns=['Transported'])
print(X_to_test.shape)

# Asegurándonos de que las variables categóricas estén codificadas como 'category'
categorical_columns = X_to_test.select_dtypes(include=['object']).columns  # Selecciona columnas categóricas

for col in categorical_columns:
    X_to_test[col] = X_to_test[col].astype('category')

# Reemplazar espacios en los nombres de las características (por si hay alguno)
X_to_test.columns = X_to_test.columns.str.replace(' ', '_')

(4277, 13)


In [1055]:
df_test.shape

(4277, 14)

In [1056]:

y_pred_test = voting_clf.predict(X_to_test)

print(df_test.shape)


passenger_ids = X_to_test['PassengerId']

print(passenger_ids.shape)

# Transformar las predicciones a True/False si es necesario
y_pred_test = y_pred_test.astype(bool)

print(y_pred_test)

# Crear un DataFrame con las columnas 'PassengerId' y 'Transported'
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Transported': y_pred_test
})

submission.head(10)

submission.shape

(4277, 14)
(4277,)
[False False  True ...  True  True False]


(4277, 2)

In [1057]:

# Guardar las predicciones en un nuevo archivo CSV
submission.to_csv('../data/titanic_submission.csv', index=False)

print(f"\n{Fore.CYAN}Archivo{Fore.RESET} {Fore.GREEN}titanic_submission.csv {Fore.CYAN}creado con éxito.{Fore.RESET}")


Archivo titanic_submission.csv creado con éxito.
